# Import and preprocess data  

Load data

In [6]:
features=np.load('original_features.npy')
adj=np.load('adj.npy')
length=np.load('length.npy')
activity=np.load('activity.npy')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(activity,bins=10)
plt.show()

Normalize adjacency matrix

In [ ]:
import scipy.sparse as sp
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [9]:
for i in range(adj.shape[0]):
    adj[i][:length[i],:length[i]]=adj[i][:length[i],:length[i]]+np.eye(length[i])
    adj[i]=normalize(adj[i])

In [10]:
features=torch.tensor(features)
adj=torch.tensor(adj)
length=torch.tensor(length)
train_x=features[:500]
train_adj=adj[:500]
train_length=length[:500]
valid_x=features[500:600]
valid_adj=adj[500:600]
valid_length=length[500:600]

In [11]:
from sklearn import preprocessing
activity=activity.reshape([activity.shape[0],1])
train_activity=activity[:500]
valid_activity=activity[500:600]
scaler = preprocessing.StandardScaler().fit(train_activity)
train_y = scaler.transform(train_activity)
valid_y=scaler.transform(valid_activity)
train_y=torch.tensor(train_y)
valid_y=torch.tensor(valid_y)



# GCN model

GCN layer

In [21]:
import math
import numpy as np
import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        output=torch.zeros(input.shape[0],input.shape[1],self.out_features)
        for i in range(input.shape[0]):
            support = torch.mm(input[i].double(), self.weight.double())
            output[i] = torch.spmm(adj[i], support.double())
            if self.bias is not None:
                for j in range(input.shape[1]):
                    output[i,j]=output[i,j] + self.bias
    
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class ave_pooling(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self):
        super(ave_pooling, self).__init__()

    
    def forward(self,input,length):
        output=torch.zeros([input.shape[0],input.shape[-1]])
        for i in range(input.shape[0]):
            output[i]=input[i,:int(length[i]),:].mean(0)
        
        return output
    
    def __repr__(self):
        return self.__class__.__name__ 

GCN model

In [22]:
import torch.nn as nn
import torch.nn.functional as F



class GCN(nn.Module):
    def __init__(self, nfeat, nhid1,nhid2,dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1)
        self.gc2 = GraphConvolution(nhid1,nhid2 )
        self.ave_pooling=ave_pooling()
        self.dropout = dropout
        self.linear=torch.nn.Linear(nhid2,1)

    def forward(self, x, adj,length):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc2(x, adj))
        #print(x[0,:,0])
        #print(x[0,:int(length[0]),0].mean())
        x=self.ave_pooling(x,length)
        x=self.linear(x)
        #print(y)
        #x=self.linear(y)
        #print(x)
        #print(x)
        #
        #
        #y=self.ave_pooling(x,length)
       # y=torch.zeros(x.shape[0],x.shape[-1])
        #for i in range(x.shape[0]):
          #  y[i]=x[i,:int(length[i]),:].mean(0)
        return x

In [23]:
GCN = GCN(nfeat = 4, nhid1=100,nhid2=50,dropout=0.3)

In [24]:
print(GCN)

GCN(
  (gc1): GraphConvolution (4 -> 100)
  (gc2): GraphConvolution (100 -> 50)
  (ave_pooling): ave_pooling
  (linear): Linear(in_features=50, out_features=1, bias=True)
)


Train model

In [25]:
import torch.utils.data as Data
torch_dataset = Data.TensorDataset(train_x,train_adj,train_length, train_y)
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=1,      # mini batch size
    shuffle=True,                             
)

In [17]:
import time
optimizer = torch.optim.Adam(GCN.parameters(), lr=0.001)  
loss_func = torch.nn.MSELoss()     

start = time.time()
hist_train_loss = []
hist_valid_loss=[]
for t in range(20):
    print(t)
    for step, (batch_x,batch_adj,batch_length, batch_y) in enumerate(loader):
        #print('step:',step)
        prediction = GCN(batch_x,batch_adj,batch_length)
        #print(prediction,batch_y)
        loss = loss_func(prediction, batch_y.float())    

        optimizer.zero_grad()   
        loss.backward()         
        optimizer.step()
    prediction_train=GCN(train_x,train_adj,train_length)
    loss_train=loss_func(prediction_train, train_y.float()) 
    prediction_valid = GCN(valid_x,valid_adj,valid_length)
    loss_valid = loss_func(prediction_valid, valid_y.float())   
    hist_train_loss.append(loss_train.data.cpu().numpy())
    hist_valid_loss.append(loss_valid.data.cpu().numpy())
    print('loss: ',loss_train.data.cpu().numpy(),'valid_loss:', loss_valid.data.cpu().numpy())
    print(time.time()-start)
torch.synchronize()
print(time.time()-start)

0
loss:  9.771044 valid_loss: 12.7822
590.6756973266602
1
loss:  19.945347 valid_loss: 19.508463
1244.0213131904602
2


KeyboardInterrupt: 